<a href="https://colab.research.google.com/github/bipana06/Activity1_Travel_Assistant/blob/main/Activity1_Travel_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Features: Travel Assistant

This is a simple data feature to help travelers have information about flights and the currency exchange rates at destination compared to thier departing/home currency.

I have used following APIs:
*   API to find the flights and associated information from a location to another.
*   API to exchange currencies.


### Steps:
1. User gives information about the departure location, destination, intended flight date etc.
2. The program uses an API to find cheap flights, the aircraft details and the flight duration.
3. Another currency converting API is used to then convert the flight costs to the local currency for the ease of the user.

### Setting up The APIs:

We need API keys to be able to use the APIs. Hence, to get an API key for amaDEUS API visit this link (https://developers.amadeus.com/get-started/get-started-with-self-service-apis-335) and create an account. We also a need a API Secret for amaDEUS which can be found right below the API key.

https://currencybeacon.com/register Visit this link and create an account to get an API key for currencybeacon.

After the API keys and the API secret are accuired, add add them to the secrets.

In [1]:
# Setting up the API keys
from google.colab import userdata

amadeus_API_KEY = userdata.get('amadeus_API_KEY')
weatherbit_API_KEY = userdata.get('weatherbit_API_KEY')
currency_API_KEY = userdata.get('currency_API_KEY')
amadeus_SECRET_KEY = userdata.get('amadeus_SECRET_KEY')

# Confirmation messages
if amadeus_API_KEY:
    print("Amadeus key successfully retrived.")
else:
    print("Retriving Amadeus key, Fail! Recheck.")

if weatherbit_API_KEY:
    print("Weather key successfully retrived.")
else:
    print("Retriving Weather key, Fail! Recheck.")

if currency_API_KEY:
    print("Currency key successfully retrived.")
else:
    print("Retriving Currency key, Fail! Recheck.")

if amadeus_SECRET_KEY:
    print("Amadeus secret key successfully retrived.")
else:
    print("Retriving Amadeus secret key, Fail! Recheck.")


Amadeus key successfully retrived.
Weather key successfully retrived.
Currency key successfully retrived.
Amadeus secret key successfully retrived.


## Currency Exchange

Here, I have used the currencybeacon API to convert an amount of a certain currency to the user desired currency. It also displays the exchange rate and the date on which the conversion is done.

In [2]:
import requests

def currency_conversion(base_currency, destination_currency, amount):

  url = f'https://api.currencybeacon.com/v1/convert?api_key={currency_API_KEY}&from={base_currency}&to={destination_currency}&amount={amount}'

  response = requests.get(url)

  if response.status_code == 200:
        data = response.json()

        converted_amount = data.get("response", {}).get("value") # get the converted amount
        exchange_rate = converted_amount / amount  # calculating the exchange rate as the API doesn't give it.
        date = data.get("response", {}).get("date") #date for the reference as to when the conversion was made

        return converted_amount, exchange_rate, date
  else:
        print("""Please check your currencies. We might not have the currency in our
                 system if your inputs are correct. Sorry for the inconvineience.""")
        return None

# Test Case
'''base_currency = "USD"
destination_currency = "AED"
amount = 1000'''

base_currency = input("Enter your the currency you want to exchange: ")
destination_currency = input("Enter the currency you want to exchange to: ")
amount = int(input("Enter the amount: "))

amount_rate_date = currency_conversion(base_currency, destination_currency, amount)

print(f"\n{amount} {base_currency} is equal to {amount_rate_date[0]} {destination_currency}")
print(f"Rate: 1 {base_currency} = {amount_rate_date[1]} {destination_currency}")
print(f"Date: {amount_rate_date[2]}")



Enter your the currency you want to exchange: AED
Enter the currency you want to exchange to: USD
Enter the amount: 4000

4000 AED is equal to 1089.17632 USD
Rate: 1 AED = 0.27229408 USD
Date: 2024-09-11


## Finding flight options from departure to a destination

Firstly, here as the API requires an access token, firstly a function is created which generates an access token using the API key and API Secret. Then the access token is used to search for flights.

When the user searches for flights, with the help of Currency Exchange API (the function written above), the flight cost can be converted to the local currency too.

In [3]:

import requests

# Get access token.... required to request flight offers
def get_access_token(amadeus_API_KEY, amadeus_SECRET_KEY):

    url = 'https://test.api.amadeus.com/v1/security/oauth2/token'
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    data = {'grant_type': 'client_credentials','client_id': amadeus_API_KEY,
            'client_secret': amadeus_SECRET_KEY}

    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
      return response.json().get('access_token')
    else:
      return None


# Get flight offers
def available_flight_offers(access_token, origin, destination, departure_date, adults, travel_class):
    url2 = 'https://test.api.amadeus.com/v2/shopping/flight-offers'
    headers2 = {'accept': 'application/vnd.amadeus+json','Authorization': f'Bearer {access_token}'}
    params = {'originLocationCode': origin,'destinationLocationCode': destination,
               'departureDate': departure_date,'adults': adults, 'travelClass': travel_class, 'max' : 7}

    response2 = requests.get(url2, params=params, headers=headers2)

    if response2.status_code == 200:

      return response2.json()
    else:
      return None


#test case
token = get_access_token(amadeus_API_KEY, amadeus_SECRET_KEY)

'''origin = 'DXB'
destination = 'JFK'
departure_date = '2024-09-25'
adults = 2
travelclass = "ECONOMY" '''

origin = input("Enter your departure location. ")
destination = input("Enter your destination. ")
departure_date = input("When do you wish to travel?(yyyy-mm-dd): ")
adults = input("No. of travelers: ")
travelclass = input("Travel class: ")
local_currency = input("Please enter your local currency (USD/AED...): ")


flight_offers = available_flight_offers(token, origin, destination, departure_date, adults, travelclass)

if flight_offers:
    print(f"\nAvailable best offers for your flight from {origin} to {destination} departing on {departure_date} are: (Price in local currency) ")
    for offers in flight_offers['data']:
      total_price = offers['price']['total']
      currency = offers['price']['currency']
      total_flight_duration = offers['itineraries'][0]['duration']
      flight_code = offers['itineraries'][0]['segments'][0]['carrierCode']
      aircraftcode = offers['itineraries'][0]['segments'][0]['aircraft']['code']

      #integration of the currency beacon API to the Amedeus API

      flight_cost_in_local_currency = currency_conversion(currency, local_currency, float(total_price))
      print(f"Price: {round(flight_cost_in_local_currency[0], 2)} {local_currency}, Airlines: {flight_code} No. {aircraftcode}, Flight duration: {total_flight_duration}")
else:
      print("Errors or No flight offers available.")



Enter your departure location. DXB
Enter your destination. JFK
When do you wish to travel?(yyyy-mm-dd): 2024-12-12
No. of travelers: 2
Travel class: BUSINESS
Please enter your local currency (USD/AED...): AED

Available best offers for your flight from DXB to JFK departing on 2024-12-12 are: (Price in local currency) 
Price: 14852.55 AED, Airlines: MS No. 789, Flight duration: PT25H40M
Price: 15778.71 AED, Airlines: MS No. 32N, Flight duration: PT23H
Price: 16248.5 AED, Airlines: AZ No. 32Q, Flight duration: PT20H25M
Price: 16248.5 AED, Airlines: AZ No. 32Q, Flight duration: PT21H35M
Price: 18006.71 AED, Airlines: AY No. 333, Flight duration: PT27H20M
Price: 20129.5 AED, Airlines: SV No. 333, Flight duration: PT19H
Price: 20129.5 AED, Airlines: SV No. 320, Flight duration: PT21H
